In [3]:
import pandas as pd
from IPython.display import Markdown, display

# 自分のチームID、出場停止チームのID
TEAM_ID = "16"
IGNORE_TEAMS = ["13"]
FILE_PATH = f"./E{TEAM_ID}_merge_election.csv"

In [4]:
def printmd(string):
    display(Markdown(string))

def validate_contest_data(file_path, team_id_to_ignore):
    ok = True
    # データの読み込み
    df = pd.read_csv(file_path, header=None)
    
    ignore_list = [int(team_id_to_ignore) - 1]
    for id in IGNORE_TEAMS:
        ignore_list.append(int(id) - 1)
    
    # テーブルが50行×44列であるかを検証
    if df.shape != (50, 44):
        print("エラー：データのサイズが50行×44列ではありません。")
        print(f"実際のサイズ：{df.shape[0]}行 × {df.shape[1]}列")
        return
    
    printmd("データサイズは50行×44列です。")
    
    # 列番号を0から始まるインデックスとして扱う
    num_columns = df.shape[1]
    
    # チームの列ペアを作成（TeamID*2, TeamID*2 + 1）
    team_columns = [(i, i+1) for i in range(0, num_columns, 2)]
    
    # 自チームと出場停止チームの列を無視する
    columns_to_check = []
    for team_idx, (col_even, col_odd) in enumerate(team_columns):
        if team_idx in ignore_list:
            continue
        columns_to_check.append((col_even, col_odd))
    
    # 偶数番目の列の値が0～49の離散値であるかを検証
    for col_even, col_odd in columns_to_check:
        team = int(col_even/2) + 1
        even_values = df[col_even].dropna()
        if not even_values.apply(lambda x: x in range(0, 50)).all():
            print(f"エラー：列 {col_even} （team:{team}）の値が0～49の範囲外の値を含んでいます。")
    
    printmd("マッチングの値は全て0～49の範囲内です。")
    
    # 奇数番目の列の値が0～5の離散値であるかを検証
    for col_even, col_odd in columns_to_check:
        team = int(col_even/2) + 1
        odd_values = df[col_odd].dropna()
        if not odd_values.apply(lambda x: x in range(0, 6)).all():
            print(f"エラー：列 {col_odd} （tean:{team}）の値が0～5の範囲外の値を含んでいます。")
    
    printmd("穴埋めの値は全て0～5の範囲内です。")
    
    # 各チームの奇数番目の列について、0の数が30個以下であるかを検証
    for _, (col_even, col_odd) in enumerate(columns_to_check):
        team_idx = int(col_even/2) + 1
        odd_values = df[col_odd]
        zero_count = (odd_values == 0).sum()
        if zero_count > 30:
            ok = False
            printmd(f"チーム {team_idx}（<span style='color:yellow'>col.{col_odd}</span>）は、<span style='color:yellow'>0の数が30個を超えています。</span>（{zero_count}個）")
        # else:
        #     print(f"チーム {team_idx}（col.{col_odd}）は、0の数が30個以下です。（{zero_count}個）")
            
    if ok:
        printmd("<span style='color:cyan'>提出可能なデータです。</span>")
    else:
        printmd("<span style='color:yellow'>提出可能なデータではありません。</span>")

validate_contest_data(FILE_PATH, TEAM_ID)

データサイズは50行×44列です。

マッチングの値は全て0～49の範囲内です。

穴埋めの値は全て0～5の範囲内です。

<span style='color:cyan'>提出可能なデータです。</span>